# Video Summarizer 모델 학습/평가 코드

In [1]:
import os
import math
import numpy as np

## 경로 설정

In [2]:
dataset_dir = r'E:\Work\YasuoNet\data\dataset14_sl3_vsr2_vw64_vh64_asr22050_mfcc'
ckpt_dir = 'checkpoints'

## 데이터 로더 생성

In [3]:
from data_loader import DataLoader

data_loader = DataLoader(dataset_dir, x_includes=['video', 'audio'])

data_config = data_loader.get_metadata()['config']
input_shape_dict = data_loader.get_metadata()['data_shape']
class_counts = data_loader.all_segment_df['label'].value_counts(sort=False)

## 하이퍼파라미터 설정

In [4]:
learning_rate = 1e-3
epochs = 20
batch_size = 256
class_weights = (1, 8)

## 모델 생성

In [5]:
from tensorflow.keras.layers import Dense, Dropout, Conv3D, Conv2D, Input, MaxPool3D, MaxPool2D, Flatten, Activation, concatenate
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

def build_model(input_shape_dict):
    video_input_shape = input_shape_dict['video']
    audio_input_shape = input_shape_dict['audio']
    weight_decay = 0.005

    # Video 3D Conv layers
    video_input = Input(video_input_shape)
    x = Conv3D(8, (3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(weight_decay))(video_input)
    x = MaxPool3D((2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    video_output = Flatten()(x)

    # Audio 2D Conv layers
    audio_input = Input(audio_input_shape)
    x = expand_dims(audio_input)    # add channel dim
    x = Conv2D(4, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPool2D((2, 2), strides=(2, 2), padding='same')(x)
    audio_output = Flatten()(x)

    # Fully-connected layers
    fc_input = concatenate([video_output, audio_output])
    x = Dense(16, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(weight_decay))(fc_input)
    #     x = Dropout(0.2)(x)
    fc_output = Dense(1, activation='sigmoid', kernel_initializer='he_uniform', kernel_regularizer=l2(weight_decay))(x)

    model = Model(inputs=[video_input, audio_input], outputs=fc_output)

    return model

In [6]:
model = build_model(input_shape_dict)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 40, 130)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 6, 64, 64, 3 0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 40, 130, 1)] 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 6, 64, 64, 8) 656         input_1[0][0]                    
______________________________________________________________________________________________

## 모델 학습

In [7]:
from trainer import Trainer
from tensorflow.keras.optimizers import Adam

# 학습 시작
trainer = Trainer(model, data_loader, ckpt_dir)
trainer.train(Adam(learning_rate), epochs, batch_size, class_weights)

Training started at 20200818-133955
optimizer: {'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
epochs: 20
batch size: 256
class weights: (1, 8)
normalized class weights: [0.61485043 4.91880342]




model saved to checkpoints\ckpt-20200818-133955-0001-0.3739.h5



model saved to checkpoints\ckpt-20200818-133955-0002-0.4239.h5



Train stopped

Top5 models
           loss  accuracy  precision    recall   f1score                        checkpoint
epoch                                                                                     
2      0.413656  0.883421   0.341719  0.558219  0.423927  ckpt-20200818-133955-0002-0.4239
1      0.516221  0.853684   0.278524  0.568493  0.373874  ckpt-20200818-133955-0001-0.3739


##  가중치 복원
모델이 선언되어 있을 때 저장된 가중치를 복원

In [9]:
checkpoint_name = 'ckpt-20200818-133955-0002-0.4239'
model.load_weights(os.path.join(ckpt_dir, checkpoint_name + '.h5'))

## 모델 테스트

In [10]:
loss, accuracy, precision, recall, f1score = trainer.test(batch_size)
print(f'loss: {loss:.4f}, accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1score: {f1score:.4f}')


loss: 0.3062, accuracy: 0.8558, precision: 0.4003, recall: 0.6708, f1score: 0.5014


In [11]:
from sklearn.metrics import confusion_matrix, classification_report

y_true, y_pred = trainer.test_prediction(batch_size)

print(f'test data count: {len(y_true)}')
print(f'true_1, pred_1: {y_true.sum(), y_pred.sum()}')
print()
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))
print()
print('Report:')
print(classification_report(y_true, y_pred))

test data count: 3711
true_1, pred_1: (401, 672)

Confusion Matrix:
[[2907  403]
 [ 132  269]]

Report:
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      3310
           1       0.40      0.67      0.50       401

    accuracy                           0.86      3711
   macro avg       0.68      0.77      0.71      3711
weighted avg       0.90      0.86      0.87      3711



## 모델의 모든 정보를 온전하게 저장 / 복원
모델의 가중치 뿐만아니라 모든 레이어 구성 정보를 저장하여 추후 모델 선언부가 없어도 불러와서 사용 가능

### 모델 저장

In [ ]:
checkpoint_name = 'ckpt-20200818-124333-0011-0.3412'
model_name = checkpoint_name + '_model'
model_path = os.path.join(ckpt_dir, model_name + '.h5')
print(model_path)

In [ ]:
model.save(model_path)

# 모델 복원

In [ ]:
checkpoint_name = 'ckpt-20200818-124333-0011-0.3412'
model_name = checkpoint_name + '_model'
model_path = os.path.join(ckpt_dir, model_name + '.h5')
print(model_path)

In [ ]:
from tensorflow.keras.models import load_model

model_restored = load_model(model_path)

In [ ]:
model_restored.summary()

In [ ]:
from trainer import Trainer
trainer = Trainer(model_restored, data_loader, ckpt_dir)

loss, accuracy, precision, recall, f1score = trainer.test(batch_size)
print(f'loss: {loss:.4f}, accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1score: {f1score:.4f}')